In [1]:
import numpy as np
import pandas as pd

In [63]:
df = pd.read_csv("/kaggle/input/spotify-million-song-dataset/spotify_millsongdata.csv", delimiter=",")
df2 = pd.read_csv("/kaggle/input/spotify-12m-songs/tracks_features.csv")

In [70]:
song_to_id_dict = {}
song_names = list(df2["name"])
song_ids = list(df2["id"])

for index in range(len(song_names)):
    song_to_id_dict[song_names[index]] = song_ids[index]

In [85]:
df_new_col = df
df_new_col["song_id"] = df["song"].apply(lambda x: song_to_id_dict[x] if x in song_to_id_dict else np.nan)
df_new_col = df_new_col.dropna(axis=0)

In [5]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

porter = PorterStemmer()
stop = stopwords.words('english')

def text_processing(text):
    text = [porter.stem(word) for word in text.split()]
    text_filter = [word for word in text if word not in stop]
    return text_filter

df["text"] = df["text"].apply(text_processing)
df_new_col["text"] = df_new_col["text"].apply(text_processing)

In [116]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
common_texts = df["text"].values
artists = df["artist"].values
songs = df["song"].values

documents = [TaggedDocument(doc, [songs[i] + "|" + artists[i]]) for i, doc in enumerate(common_texts)]

common_texts_id = df_new_col["text"].values
artists_id = df_new_col["artist"].values
song_id = df_new_col["song_id"].values
documents_id = [TaggedDocument(doc, [song_id[i]]) for i, doc in enumerate(common_texts_id)]

In [117]:
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, workers=4, epochs=20)
model_id = Doc2Vec(documents_id, vector_size=100, window=5, min_count=1, workers=4, epochs=20)

In [118]:
test = """Ribeye steaks are mostly composed of the longissimus dorsi muscle but also contain the complexus and spinalis muscles. The longissimus dorsi is also referred to as the "eye of the ribeye". The spinalis is also referred to as the "ribeye cap" and the complexus is a small muscle at the front of the ribeye which may be trimmed off by the butcher."""

test = text_processing(test)
new_doc_vector = model.infer_vector(test)
similar_documents = model.dv.most_similar(new_doc_vector, topn=5)

new_doc_vector_id = model_id.infer_vector(test)
similar_documents_id = model_id.dv.most_similar(new_doc_vector, topn=5)

In [122]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
